<a href="https://colab.research.google.com/github/fcoliveira-utfpr/agrometeorologia/blob/main/bh_cultura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Inserindo informações**
---

In [83]:
municipio = "Santa Helena" ###### Nome aqui entre aspas

estado = "PR" ###### Sigla do estado entre aspas

cultura = "Milho" ## Nome da cultura

ano_inicial = 2020  #Ano da semeadura

#datas de semeadura************************************************************
sem = ['20/05/']  #Datas de semeadura

# **Importa das bibliotecas**
---

In [84]:
#importantando bibliotecas
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import json
import requests
import os
import openpyxl
from google.colab import files
import matplotlib.pyplot as plt

# **Encontrando a DTA do solo**
---

In [85]:
url_1 = "https://raw.githubusercontent.com/fcoliveira-utfpr/agrometeorologia/refs/heads/main/clima_solo_local.csv"
df1 = pd.read_csv(url_1)
cidade = df1['Município']
uf = df1['Estado']
regiao = df1['Região']
koppen = df1['Köppen']
df1_valores = df1.drop(columns=['Município', 'Estado','Região',	'Köppen'])
df1_valores = df1_valores.replace({',': '.'}, regex=True)
df1_valores = df1_valores.apply(pd.to_numeric, errors='coerce')
df1 = df1_valores.copy()
df1['Município'] = cidade
df1['Estado'] = uf
df1['Região'] = regiao
df1['Köppen'] = koppen
#df1

# **Obtendo dados da cultura**
---

In [86]:
url_2 = "https://raw.githubusercontent.com/fcoliveira-utfpr/agrometeorologia/refs/heads/main/dados_culturas.csv"
df2 = pd.read_csv(url_2)
cult = df2['Cultura']
df2 = df2.drop(columns=['Cultura'])
df2_valores = df2.replace({',': '.'}, regex=True)
df2_valores = df2_valores.apply(pd.to_numeric, errors='coerce')
df2 = df2_valores.copy()
df2['Cultura'] = cult
df2 = df2[(df2['Cultura'] == cultura)].copy()
#df2

In [87]:
df11 = df1[(df1['Município'] == municipio) & (df1['Estado'] == estado)].copy()

DTA = df11['DTA (mm/m)'].iloc[0]  # mm/m

duracao_do_ciclo = 120  # dias

# Fases fenológicas (%)
fase1 = df2['F1 %'].iloc[0]  # %
fase2 = df2['F2 %'].iloc[0]  # %
fase3 = df2['F3 %'].iloc[0]  # %
fase4 = df2['F4 %'].iloc[0]  # %

# Coeficientes de cultura (Kc)
kc1 = df2['Kc ini'].iloc[0]    # fase inicial
kc2 = df2['Kc méd'].iloc[0]    # fase intermediária (máximo desenvolvimento)
kc3 = df2['Kc fin'].iloc[0]   # fase final (maturação)

# Profundidade das raízes (Z)
z_inicial = 0.05  # m
z_final = df2['Z efetivo (m)'].iloc[0]

nome_cidade = df11["Município"].iloc[0]
latitude = df11["latitude"].iloc[0]
longitude = df11["longitude"].iloc[0]
alt = df11["Altitude"].iloc[0]

anos = list(range(ano_inicial, ano_final+1))

# **Download NASA POWER**
---

In [89]:
# ============================================================
# Download NASA POWER
# ============================================================

base_dados = 'NASAPOWER'

ano_final = ano_inicial+1
# Datas de início e fim do período
data_ini = datetime(ano_inicial, 1, 1).date()
data_fim = datetime(ano_final, 12, 31).date()

ini = int(data_ini.strftime('%Y%m%d'))
fin = int(data_fim.strftime('%Y%m%d'))

# Parâmetros:
# T2M_MIN: temperatura mínima
# T2M_MAX: temperatura máxima
# PRECTOTCORR: precipitação corrigida
# RH2M: umidade relativa
# WS2M: vento a 2m
# ALLSKY_SFC_SW_DWN: radiação solar global
# TOA_SW_DWN: radiação extraterrestre
base_url = (
    "https://power.larc.nasa.gov/api/temporal/daily/point"
    "?parameters=T2M_MIN,T2M_MAX,PRECTOTCORR,RH2M,WS2M,ALLSKY_SFC_SW_DWN,TOA_SW_DWN"
    "&community=RE"
    "&longitude={longitude}"
    "&latitude={latitude}"
    "&start={ini}"
    "&end={fin}"
    "&format=JSON"
)

api_request_url = base_url.format(
    longitude=longitude,
    latitude=latitude,
    ini=ini,
    fin=fin
)

response = requests.get(url=api_request_url, verify=True, timeout=120)
response.raise_for_status()

dados_json = response.json()
propriedades = dados_json['properties']

# A parte importante está em propriedades['parameter']
parametros = propriedades['parameter']

datas_str = sorted(list(parametros['T2M_MIN'].keys()))
datas = [pd.to_datetime(d) for d in datas_str]

# Constrói o DataFrame básico com a coluna Data
df = pd.DataFrame({'Data': datas})

# Temperatura mínima
df['Tmin'] = [parametros['T2M_MIN'][d] for d in datas_str]

# Temperatura máxima
df['Tmax'] = [parametros['T2M_MAX'][d] for d in datas_str]

# Chuva (PRECIPITAÇÃO CORRIGIDA)
df['Chuva'] = [parametros['PRECTOTCORR'][d] for d in datas_str]

# Umidade relativa
df['UR'] = [parametros['RH2M'][d] for d in datas_str]

# Vento médio a 2m
df['U2'] = [parametros['WS2M'][d] for d in datas_str]

# Radiação solar global (Rs) e extraterrestre (Qo), em kWh/m²d (converteremos depois)
df['Rs_raw'] = [parametros['ALLSKY_SFC_SW_DWN'][d] for d in datas_str]
df['Qo_raw'] = [parametros['TOA_SW_DWN'][d] for d in datas_str]

# Converter Rs e Qo para MJ/m²/dia (kWh * 3.6)
df['Rs'] = df['Rs_raw'] * 3.6
df['Qo'] = df['Qo_raw'] * 3.6

# Temperatura média
df['Tmed'] = (df['Tmax'] + df['Tmin']) / 2

# Pressão atmosférica (Patm) – depende da altitude
df['Patm'] = 101.3 * ((293 - 0.0065 * alt) / 293) ** 5.26

# Número do dia do ano (NDA)
df['NDA'] = [pd.to_datetime(d).timetuple().tm_yday for d in df['Data']]

# Declinação solar
def dec_sol(NDA):
    return 23.45 * np.sin(np.deg2rad(360/365 * (NDA - 80)))

df['d'] = df['NDA'].apply(dec_sol)

# Hora do nascer do sol (Hn), em graus (igual ao professor)
def hora_na_sol(d, Lat):
    return np.rad2deg(np.arccos(-(np.tan(np.deg2rad(Lat)) * np.tan(np.deg2rad(d)))))

df['Hn'] = df['d'].apply(lambda d_: hora_na_sol(d_, latitude))

# Relação (d/D)^2 (correção da distância Terra-Sol)
def relacao_d_D_2(NDA):
    return 1 + 0.033 * np.cos(np.deg2rad(NDA * 360/365))

df['(d/D)²'] = df['NDA'].apply(relacao_d_D_2)

# BOC (balanço de onda curta: Rs * (1 - 25%))
df['BOC'] = df['Rs'] * (1 - 25/100)

# Função de pressão de saturação e
def e_saturacao(temp):
    return 0.6108 * 10 ** ((7.5 * temp) / (237.3 + temp))

# es_Tmax, es_Tmin, es médio
df['es_Tmax'] = df['Tmax'].apply(e_saturacao)
df['es_Tmin'] = df['Tmin'].apply(e_saturacao)
df['es'] = (df['es_Tmax'] + df['es_Tmin']) / 2

# ea – pressão parcial de vapor
df['ea'] = (df['UR'] / 100.0) * df['es']

# QGcs (radição em céu claro)
df['QGcs'] = df['Qo'] * (0.75 + (2e-5) * alt)

# BOL (balanço de onda longa)
def BOL(tmax, tmin, eaa, qg, qgcs):
    a = 4.903e-9 * (((tmax + 273.16)**4 + (tmin + 273.16)**4) / 2)
    b = (0.34 - 0.14 * np.sqrt(eaa))
    c = 1.35 * (qg / qgcs) - 0.35
    return -(a * b * c)

df['BOL'] = df.apply(lambda x: BOL(x['Tmax'], x['Tmin'], x['ea'], x['Rs'], x['QGcs']), axis=1)

# Rn – saldo de radiação
df['Rn'] = df['BOC'] + df['BOL']

# s – declividade da curva de saturação
df['s'] = (4098 * df['es']) / (df['Tmed'] + 237.3) ** 2

# gama – constante psicrométrica
df['gama'] = (0.665e-3) * df['Patm']

# ETo (Penman-Monteith)
def ETo(s, Rn, gama, u2, es, ea, tmed):
    ETo1 = 0.408 * s * Rn
    ETo2 = (gama * 900 * u2 * (es - ea)) / (tmed + 273)
    ETo3 = s + gama * (1 + 0.34 * u2)
    return (ETo1 + ETo2) / ETo3

df['ETo'] = df.apply(lambda x: ETo(x['s'], x['Rn'], x['gama'], x['U2'], x['es'], x['ea'], x['Tmed']), axis=1)

# Metadados da cidade
df['Municipio'] = nome_cidade
df['Lat'] = latitude
df['Lon'] = longitude
df['Alt'] = alt

#**Balanço hídrico de cultura**
---

In [90]:
#cálculos iniciais--------------------------------------------------------------
fasetotal = np.sum(fase1 + fase2 + fase3 + fase4) #soma das fases em porcentagem

#fases fenológicas (dias)
fase1d = ((duracao_do_ciclo * fase1)/fasetotal) #dias
fase2d = ((duracao_do_ciclo * fase2)/fasetotal) #dias
fase3d = ((duracao_do_ciclo * fase3)/fasetotal) #dias
fase4d = ((duracao_do_ciclo * fase4)/fasetotal) #dias
fasetotald = np.sum(fase1d + fase2d + fase3d + fase4d) #dias

acrescimo_kc1  = (kc2 - kc1)/fase2d
acrescimo_kc2 = (kc3 - kc2)/fase4d

acrescimo_z = (z_final - z_inicial)/(fase1d + fase2d)#--------------------------

dias = []
for i in sem:
  dias.append(f'{i}{ano}')
dias
ai = []
for i in dias:
  ai.append(datetime.strptime(i,'%d/%m/%Y').date())
ai
df1 = pd.DataFrame({
                        'Datas de semeadura': ai,
                      })
mat = []
for i in df1['Datas de semeadura']:
  mat.append(i +  timedelta(days=duracao_do_ciclo-1))
df1['Datas maturação'] = mat
din = []
for i in df1['Datas de semeadura']:
  din.append(pd.to_datetime(i))
df1['Datas de semeadura'] = din

dfn = []
for i in df1['Datas maturação']:
  dfn.append(pd.to_datetime(i))
df1['Datas maturação'] = dfn

In [100]:
#**************SIMULAÇÃO DE PRODUTIVIDADE**************************************************
safra1 = (df.loc[(df['Data'] >= df1['Datas de semeadura'][0]) & (df['Data'] <= df1['Datas maturação'][0])]).copy()

safra1['n'] = list(range(1, safra1['NDA'].count() +1))

# -------------------------------
# 1) Kc e FASE da cultura (igual prof)
# -------------------------------
fase_cultura = []
Kc1 = []

for n in safra1['n']:
    if n <= fase1d:
        # Fase 1: Kc = kc1
        Kc1.append(kc1)
        fase_cultura.append(1)

    elif n <= fase1d + fase2d:
        # Fase 2: crescimento linear kc1 -> kc2
        Kc1.append(Kc1[-1] + acrescimo_kc1)
        fase_cultura.append(2)

    elif n <= fase1d + fase2d + fase3d:
        # Fase 3: Kc constante = kc2
        Kc1.append(kc2)
        fase_cultura.append(3)

    else:
        # Fase 4: decaimento linear kc2 -> kc3
        Kc1.append(Kc1[-1] + acrescimo_kc2)
        fase_cultura.append(4)

safra1['Kc'] = Kc1
safra1['Fase'] = fase_cultura

# -------------------------------
# 2) Crescimento da raiz z
# -------------------------------
z1 = []
for n in safra1['n']:
    if n == 1:
        z1.append(z_inicial + acrescimo_z)
    elif n <= fase1d + fase2d:
        z1.append(z1[-1] + acrescimo_z)
    else:
        z1.append(z_final)

safra1['z'] = z1

# -------------------------------
# 3) CAD, ETc, P-ETc
# -------------------------------
safra1['CAD'] = safra1['z'] * DTA          # CAD (mm) no perfil explorado
safra1['ETc'] = safra1['Kc'] * safra1['ETo']    # ETc diária
safra1['P-ETc'] = safra1['Chuva'] - safra1['ETc']

# -------------------------------
# 4) ARM (armazenamento de água no solo)
# -------------------------------
PETc1 = safra1['P-ETc'].to_numpy()
CAD1 = safra1['CAD'].to_numpy()

# Estado inicial de ARM (solo cheio na profundidade inicial)
ARM1 = [z_inicial * DTA]

for p, cad in zip(PETc1, CAD1):
    prev = ARM1[-1]
    if p < 0:
        # Déficit: usa equação exponencial (igual professor)
        ARM1.append(prev * np.exp(p / cad))
    elif p + prev >= cad:
        # Encharcou: limita ao CAD
        ARM1.append(cad)
    else:
        # Situação normal
        ARM1.append(prev + p)

# Remove o estado inicial e deixa só os valores correspondentes aos dias
ARM1 = ARM1[1:]
safra1['ARM'] = ARM1

# -------------------------------
# 5) ALT (variação diária de ARM)
# -------------------------------
ALT = [0]  # primeiro dia, variação zero
ALT += list(np.array(ARM1[1:]) - np.array(ARM1[:-1]))
safra1['ALT'] = ALT

# -------------------------------
# 6) ETR, DEF, EXC, ISNA diário
# -------------------------------
safra1['ETR'] = np.where(
    safra1['P-ETc'] < 0,
    safra1['Chuva'] + safra1['ALT'].abs(),  # solo complementa a chuva
    safra1['ETc']                           # sem limitação hídrica
)

safra1['DEF'] = safra1['ETc'] - safra1['ETR']

safra1['EXC'] = np.where(
    safra1['ARM'] < safra1['CAD'],
    0,
    safra1['P-ETc'] - safra1['ALT']
)

safra1['ISNA_diario'] = safra1['ETR'] / safra1['ETc']

safra1

,Data,Tmin,Tmax,Chuva,UR,U2,Rs_raw,Qo_raw,Rs,Qo,...,z,CAD,ETc,P-ETc,ARM,ALT,ETR,DEF,EXC,ISNA_diario
140,2020-05-20,24.46,29.40,7.16,93.10,0.14,5.1242,9.2803,18.44712,33.40908,...,0.056481,6.196019,1.087665,6.072335,6.196019,0.000000,1.087665,0.000000,6.072335,1.000000
141,2020-05-21,24.33,28.85,8.90,92.43,0.15,3.7855,9.2621,13.62780,33.34356,...,0.062963,6.907037,0.825515,8.074485,6.907037,0.711019,0.825515,0.000000,7.363467,1.000000
142,2020-05-22,24.63,28.51,22.02,94.01,0.22,4.1933,9.2443,15.09588,33.27948,...,0.069444,7.618056,0.899521,21.120479,7.618056,0.711019,0.899521,0.000000,20.409460,1.000000
143,2020-05-23,24.44,29.37,13.01,92.20,0.14,3.8018,9.2273,13.68648,33.21828,...,0.075926,8.329074,0.834218,12.175782,8.329074,0.711019,0.834218,0.000000,11.464763,1.000000
144,2020-05-24,23.55,32.91,0.95,83.25,0.21,5.6880,9.2102,20.47680,33.15672,...,0.082407,9.040093,1.221631,-0.271631,8.082530,-0.246544,1.196544,0.025087,0.000000,0.979465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,2020-09-12,23.70,36.93,0.04,67.94,0.14,5.8291,10.1825,20.98476,36.65700,...,0.400000,43.880000,2.960696,-2.920696,0.494541,-0.034037,0.074037,2.886659,0.000000,0.025007
256,2020-09-13,23.91,36.94,0.07,67.74,0.15,5.7470,10.1969,20.68920,36.70884,...,0.400000,43.880000,2.834099,-2.764099,0.464350,-0.030191,0.100191,2.733908,0.000000,0.035352
257,2020-09-14,22.93,35.92,0.00,67.66,0.16,6.2160,10.2108,22.37760,36.75888,...,0.400000,43.880000,2.867782,-2.867782,0.434973,-0.029377,0.029377,2.838405,0.000000,0.010244
258,2020-09-15,22.42,36.99,0.00,65.68,0.19,6.0994,10.2242,21.95784,36.80712,...,0.400000,43.880000,2.751799,-2.751799,0.408533,-0.026440,0.026440,2.725359,0.000000,0.009608
